# 6 Composition with ANCOM

We can be interested in seeing if there are differences in the composition of the communities in different body sizes, for example. To do so, we can use ANCOM.

The procedure is as follows:
- Filter the tables according to the region of interest, in this case that body-site has to be the gut

In [1]:
conda activate qiime2-2023.2

(qiime2-2023.2) 


: 1

In [3]:
qiime feature-table filter-samples \
    --i-table          table.qza \
    --m-metadata-file  sample-metadata.tsv \
    --p-where          "[body-site]='gut'" \
    --o-filtered-table gut-table.qza

Saved FeatureTable[Frequency] to: gut-table.qza
(qiime2-2023.2) 


: 1

Then, fix the table with pseudo-counts, becasue ANCOM cannot work with zeros in the table.

In [4]:
qiime composition add-pseudocount \
    --i-table             gut-table.qza \
    --o-composition-table comp-gut-table.qza

Saved FeatureTable[Composition] to: comp-gut-table.qza
(qiime2-2023.2) 


: 1

Finally, run ANCOM

In [5]:
qiime composition ancom \
    --i-table           comp-gut-table.qza \
    --m-metadata-file   sample-metadata.tsv \
    --m-metadata-column subject \
    --o-visualization   ancom-subject.qzv

Saved Visualization to: ancom-subject.qzv
(qiime2-2023.2) 


: 1

- [ancom-subject.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fancom-subject.qzv)